<a href="https://colab.research.google.com/github/AminMirzaie/SVMTutorial-MLforBio/blob/master/sort_approx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pandas==1.2.2
# !pip install sklearn numerapi
# !pip install pytorch-metric-learning
# !pip install pkbar

In [ ]:
import numpy as np
import numpy as np
import pandas as pd
import random
import numerapi
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
from sklearn.decomposition import NMF
import pkbar

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import scipy.stats
from datetime import datetime

from pytorch_metric_learning import losses



%load_ext tensorboard

In [ ]:
napi = numerapi.NumerAPI(public_id="U6GQVKDZCAX3PZCUVZU5YG4KKUZXEHKY",
                         secret_key="SHIGGP34Q7ODY5LIA55S7CT3FH4BZUPVXOIEPUVWTDY6F3S6OXH2WA7KJQ7QSHWG")

In [ ]:
napi.download_current_dataset(unzip=True)

2021-05-20 09:20:28,252 INFO numerapi.utils: starting download
./numerai_dataset_264.zip: 404MB [00:07, 53.8MB/s]                           
2021-05-20 09:20:35,776 INFO numerapi.base_api: unzipping file...


'./numerai_dataset_264.zip'

In [ ]:
class FastTensorDataLoader:
    """
    A DataLoader-like object for a set of tensors that can be much faster than
    TensorDataset + DataLoader because dataloader grabs individual indices of
    the dataset and calls cat (slow).
    Source: https://discuss.pytorch.org/t/dataloader-much-slower-than-manual-batching/27014/6
    """
    def __init__(self, *tensors, batch_size=32, shuffle=False):
        """
        Initialize a FastTensorDataLoader.
        :param *tensors: tensors to store. Must have the same length @ dim 0.
        :param batch_size: batch size to load.
        :param shuffle: if True, shuffle the data *in-place* whenever an
            iterator is created out of this object.
        :returns: A FastTensorDataLoader.
        """
        assert all(t.shape[0] == tensors[0].shape[0] for t in tensors)
        self.tensors = tensors

        self.dataset_len = self.tensors[0].shape[0]
        self.batch_size = batch_size
        self.shuffle = shuffle

        # Calculate # batches
        n_batches, remainder = divmod(self.dataset_len, self.batch_size)
        if remainder > 0:
            n_batches += 1
        self.n_batches = n_batches
    def __iter__(self):
        if self.shuffle:
            r = torch.randperm(self.dataset_len)
            self.tensors = [t[r] for t in self.tensors]
        self.i = 0
        return self

    def __next__(self):
        if self.i >= self.dataset_len:
            raise StopIteration
        batch = tuple(t[self.i:self.i+self.batch_size] for t in self.tensors)
        self.i += self.batch_size
        return batch

    def __len__(self):
        return self.n_batches

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(310, 256, bias=True)
        self.b1 = nn.BatchNorm1d(256)
        self.l2 = nn.Linear(256, 128, bias=True)
        self.b2 = nn.BatchNorm1d(128)
        self.l3 = nn.Linear(128, 64, bias=True)
        self.b3 = nn.BatchNorm1d(64)
        self.l4 = nn.Linear(64, 32, bias=True)
        self.b4 = nn.BatchNorm1d(32)
        self.l5 = nn.Linear(32, 16, bias=True)
        self.b5 = nn.BatchNorm1d(16)
        self.l6 = nn.Linear(16, 8, bias=True)
        self.b6 = nn.BatchNorm1d(8)
        self.l7 = nn.Linear(8, 1, bias=True)
        self.b7 = nn.BatchNorm1d(1)

    def forward(self, x):
        x = self.b1(F.relu(self.l1(x)))
        x = self.b2(F.relu(self.l2(x)))
        x = self.b3(F.relu(self.l3(x)))
        x = self.b4(F.relu(self.l4(x)))
        x = self.b5(F.relu(self.l5(x)))
        last = self.b6(F.relu(self.l6(x)))
        x = self.b7(F.relu(self.l7(last)))
        return x


In [ ]:
train_data = pd.read_csv("numerai_dataset_264/numerai_training_data.csv").set_index("id")

In [ ]:
feature_cols = train_data.columns[train_data.columns.str.startswith('feature')]
feature_map_dic = {}
colnames = train_data[feature_cols].columns
for i in range(310):
  feature_map_dic[i] = colnames[i]

In [ ]:
train_data["erano"] = train_data.era.str.slice(3).astype(int)
feature_cols = train_data.columns[train_data.columns.str.startswith('feature')]
train_data = train_data.astype({k: 'float32' for k in (feature_cols.to_list()+['target'])})
feature_groups = {
    g: [c for c in train_data if c.startswith(f"feature_{g}")]
    for g in ["intelligence", "wisdom", "charisma", "dexterity", "strength", "constitution"]
}

In [69]:
batch_size = 100

train_loader_dict = {}
train_loader_era_wise = {}
train_numpy_feat = []
train_numpy_label = []
for erano in train_data["erano"].unique():
  data = train_data[train_data["erano"] == erano]
  train_data_era, val_data_era = train_test_split(data, test_size=0.07, shuffle=True)
  train_loader_era_wise[erano] = {"train": FastTensorDataLoader(torch.from_numpy(train_data_era[feature_cols].values), 
                                                    torch.from_numpy(train_data_era["target"].values), 
                                                    batch_size=batch_size, shuffle=True),
                                  "val": FastTensorDataLoader(torch.from_numpy(val_data_era[feature_cols].values), 
                                                    torch.from_numpy(val_data_era["target"].values), 
                                                    batch_size=val_data_era.shape[0], shuffle=True)}
    

  

In [ ]:
# for k in range(2):
#   for i,data in enumerate(train_loader_era_wise[1]["train"],0):
#     labels = data[1]
#     data = data[0]
#     print(data.shape)

In [72]:
def softmax(a):
  return torch.exp(a) / torch.sum(torch.exp(a))

def gain(scores):
  return torch.pow(2,scores) - 1

def discount(scores):
  n = scores.shape[0]
  discount = torch.log2(torch.arange(1,n+1)+1)
  discount = discount.cuda()
  return discount
def permutation_estimator(scores,thau):
  n = scores.shape[0]
  scores = scores.reshape((n,1))
  As = torch.empty((n,n))
  As = As.cuda()
  for i in range(As.shape[0]):
    for j in range(As.shape[1]):
      As[i,j] = abs(scores[i] - scores[j])
  ones = torch.ones((n,1))
  ones = ones.cuda()
  permute = torch.zeros((n,n))
  permute = permute.cuda()
  for i in range(1,n+1):
    permute[i-1,:] = softmax(((n+1-(2*i))*scores - torch.matmul(As,ones) )/ thau).reshape(n)
  
  return permute

In [75]:
def NCDG_loss(outputs,labels,norm):
  p_hat = permutation_estimator(outputs,10)
  d = discount(labels)
  g = gain(labels)
  m = torch.mul(g,torch.matmul(p_hat,d))/norm
  loss = torch.sum(m)
  return loss

In [85]:
from sklearn.model_selection import train_test_split

startTime = datetime.now()

net = Net()
net = net.cuda()
learning_rate = 0.0001

# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
# scheduler = StepLR(optimizer, step_size=1, gamma=0.1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pbar = pkbar.Pbar(name='training', target=int(train_loader_era_wise[1]["train"].dataset_len /batch_size) )
counter = 0

for epoch in range(60):  # loop over the dataset multiple times
  counter = 0
  for i, data in enumerate(train_loader_era_wise[1]["train"], 0):
    pbar.update(counter)
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    sort_labels,_ = torch.sort(labels,descending=True)
    norm_factor = torch.sum((torch.mul(gain(sort_labels),discount(sort_labels))))
    optimizer.zero_grad()
    outputs = net(inputs)
    print(labels,outputs.reshape(outputs.shape[0]))
    loss = NCDG_loss(outputs, labels,norm_factor)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    counter+=1
  print("\n average loss = ", running_loss/train_loader_era_wise[1]["train"].dataset_len /batch_size)
  running_loss = 0.0

  with torch.no_grad():
    for i, data in enumerate(train_loader_era_wise[1]["val"], 0):
      inputs, labels = data
      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs= net(inputs)
      cor = scipy.stats.spearmanr(outputs.detach().cpu().numpy(), labels.detach().cpu().numpy())[0]
      print('Validation cor  : %f %%' % (cor))

    # if counter % 100 == 99:
      # scheduler.step()
      # print("new LR = ", scheduler.get_last_lr())
  # with torch.no_grad():
  #   running_loss = 0.0
  #   correct = 0
  #   total = 0
  #   for data in val_loader:
  #     inputs, labels = data
  #     inputs = inputs.to(device)
  #     labels = labels.to(device)
  #     outputs,_ = net(inputs)
  #     _, predicted = torch.max(outputs.data, 1)
  #     total += labels.size(0)
  #     correct += (predicted == labels).sum().item()

  #   print('Validation Accuracy: %d %%' % (100 * correct / total))
              
#     correct = 0
#     total = 0
#     for data in train_loader:
#       inputs, labels = data
#       inputs = inputs.to(device)
#       labels = labels.to(device)
#       outputs,_ = net(inputs)
#       _, predicted = torch.max(outputs.data, 1)
#       total += labels.size(0)
#       correct += (predicted == labels).sum().item()
#     print('train Accuracy: %d %%' % (100 * correct / total))
#   torch.save(net.state_dict(), "./drive/MyDrive/inv2/kotche_for_feature_"+str(feature_number))  

training
 1/22  [>.............................] - 0.0stensor([0.2500, 0.5000, 0.7500, 0.5000, 0.5000, 0.2500, 0.2500, 0.5000, 0.2500,
        0.5000, 0.5000, 0.5000, 0.5000, 1.0000, 0.5000, 0.5000, 0.7500, 0.5000,
        0.5000, 0.2500, 0.7500, 0.2500, 0.5000, 0.7500, 0.5000, 0.5000, 0.7500,
        0.7500, 0.0000, 0.5000, 0.5000, 0.2500, 0.0000, 0.5000, 0.2500, 0.5000,
        0.5000, 0.0000, 0.5000, 0.7500, 0.5000, 0.2500, 0.5000, 0.2500, 0.7500,
        0.5000, 0.5000, 0.2500, 0.0000, 0.5000, 0.5000, 0.5000, 0.7500, 0.5000,
        0.7500, 0.2500, 0.5000, 0.5000, 0.5000, 0.2500, 0.5000, 0.7500, 0.7500,
        0.5000, 1.0000, 1.0000, 0.5000, 0.5000, 0.7500, 0.2500, 0.5000, 0.0000,
        0.5000, 0.7500, 0.5000, 1.0000, 0.2500, 0.7500, 0.2500, 0.5000, 0.5000,
        0.7500, 1.0000, 1.0000, 0.5000, 0.7500, 0.7500, 0.5000, 0.5000, 0.2500,
        0.5000, 0.2500, 0.5000, 0.5000, 1.0000, 0.5000, 1.0000, 0.2500, 0.5000,
        0.7500], device='cuda:0') tensor([-0.4753,  0.6909,  0.77

KeyboardInterrupt: ignored

In [108]:
def permutation_estimator(scores,thau):
  n = scores.shape[0]
  scores = scores.reshape((n,1))
  As = torch.empty((n,n))
  As = As
  for i in range(As.shape[0]):
    for j in range(As.shape[1]):
      As[i,j] = abs(scores[i] - scores[j])
  ones = torch.ones((n,1))
  ones = ones
  permute = torch.zeros((n,n))
  permute = permute
  for i in range(1,n+1):
    a = torch.argmax(softmax(((n+1-(2*i))*scores - torch.matmul(As,ones) )/ thau).reshape(n))
    
    permute[i-1,a ] = 1.0
  return permute

def softmax(a):
  return torch.exp(a) / torch.sum(torch.exp(a))

tensor([[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]])

In [153]:
a= torch.tensor([7.0,1.0,4.0,3.0,5.6])
b = torch.tensor([2.0,7.0,0.0,1.0,12.0])
b_rank = torch.tensor([- ,2 , - , 2 , 1])
n = b.shape[0]
p_hat = permutation_estimator(a,1)
1 -(6*torch.sum(torch.abs(torch.matmul(b,p_hat) - torch.arange(1,5,dtype=torch.float32))**2))/(n*(n**2-1))

tensor(0.6000)

In [137]:

scipy.stats.spearmanr(a.detach().cpu().numpy(), b.detach().cpu().numpy())[0]


0.6000000000000001

In [129]:
rank = torch.arange(1,6,dtype=torch.float32)

In [130]:
p_hat_inv = torch.inverse(p_hat)

In [131]:
torch.matmul(torch.inverse(p_hat) , rank)

tensor([4., 5., 2., 3., 1.])